### Reformat LPR behavioral data to have t0-t239 or t0-t14

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, random, time

import warnings
warnings.filterwarnings('ignore')

In [2]:
starting_dir = os.getcwd()
print (starting_dir)

/Users/kimd999/research/script_not_in_dropbox/srpAnalytics/analysis/latest/1_reformat/all_targets/LPR


In [52]:
# mac - phase I & II - LPR - 240 endpoints
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/after_Lisa_fix/full/240_endpoints/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_w_240_endpoints.csv'
# -> 196 unique chemical IDs

# mac - phase I & II - LPR - 15 endpoints
complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/after_Lisa_fix/full/15_endpoints/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_w_15_endpoints.csv'
# -> 148 unique chemical IDs

# mac - phase III - full
#complete_input_file_path= '/Users/kimd999/research/projects/toxicity/per_each_data/phase_III/input/original/behavior/LPR/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23.csv'
# -> 240 unique variables and 215 unique chemical IDs

# constance - phase I & II - LPR - 15 endpoints
#complete_input_file_path= '/people/kimd999/tox/phase_I_II/LPR/input/tall/after_Lisa_fix/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_w_15_endpoints.csv'

# constance - phase III - full
#complete_input_file_path= '/people/kimd999/tox/phase_III/Tanguay_Phase_3_zf_LPR_data_PNNL_2021MAR23.csv'

In [75]:
df_behav = pd.read_csv(complete_input_file_path, header = 0)
df_behav = df_behav.rename({'endpoint': 'timepoint'}, axis=1)

#display(df_behav.head())

In [76]:
# Keep only relevant columns
columns_to_keep = ['chemical.id', 'conc', 'plate.id', 'well', 'timepoint', 'value']
df_select = df_behav.loc[:,columns_to_keep]

df_select['chemical.id'] = df_select['chemical.id'].astype(str)
# this recasting is needed for "df_select_1846 = df_select.loc[df_select['chemical.id'] == '1846',:]" later

display("number of unique timepoints:" + str(len(np.unique(df_select['timepoint']))))
display("number of chemical IDs:" + str(len(np.unique(df_select['chemical.id']))))

'number of unique timepoints:15'

'number of chemical IDs:148'

In [77]:
# [new] recast plate.id into string only
# this recasting is needed since
#"unique plate.id -> ['1' '1.0' '2' '2.0' 'TP962-H10-P1']"
# because of this heteronegeity, even
# display("see unique plate.id:" + str(np.unique(df_select['plate.id'])))
# is not possible

start_time = time.time()
          
def to_string(chemical_index, df, df_recasted):
    df_per_chemical = df.loc[df['chemical.id'] == str(chemical_index),:]
    
    df_per_chemical['plate.id'] = df_per_chemical['plate.id'].astype(str)

    df_recasted = pd.concat([df_recasted, df_per_chemical])
    return df_recasted
########### end of def to_string(chemical_index, df_select, df_recasted):


def recast(chemical_index, df_select, df_recasted):
    df_per_chemical = df_select.loc[df_select['chemical.id'] == str(chemical_index),:]

    # below 3 layerd recasting may seem wrong/inefficient,
    # but this is the only solution that I found that works
    # for "unique plate.id:['1' '2' 'TP962-H10-P1']"
    # (big lesson) -> recasting of int/float/str -> int is not possible
    # however, recasting of int/float -> float -> int is possible
    
    try:
        df_per_chemical['plate.id'] = df_per_chemical['plate.id'].astype(int)
    except:
        try:
            df_per_chemical['plate.id'] = df_per_chemical['plate.id'].astype(float)
        except:
            pass
    
    try:
        df_per_chemical['plate.id'] = df_per_chemical['plate.id'].astype(int)
    except:    
        df_per_chemical['plate.id'] = df_per_chemical['plate.id'].astype(str)
        
    df_per_chemical['plate.id'] = df_per_chemical['plate.id'].astype(str)
            

#     unique_plate_id_per_chem = np.unique(df_per_chemical['plate.id'])
#     for i in range(len(unique_plate_id_per_chem)):
#         display(unique_plate_id_per_chem[i])
#         display(type(unique_plate_id_per_chem[i]))
        
    df_recasted = pd.concat([df_recasted, df_per_chemical])
    return df_recasted
########### end of def recast(chemical_index, df_select, df_recasted):


df_str = pd.DataFrame()
df_recasted = pd.DataFrame()


full_devel = "full"
#full_devel = "devel"

chemical_id_from_here = np.unique(df_behav['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 1)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

#chemical_id_from_here = ['163', '1846']

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    #print("chemical_index:" + str(chemical_index))
    df_str = to_string(chemical_index, df_select, df_str)
    df_recasted = recast(chemical_index, df_str, df_recasted)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Recasting types is done. It took :"+str(time_took)) 
# took 12 seconds for minutes for 148 chemicals

display(df_recasted.head())
display(df_recasted.tail())

Recasting types is done. It took :15.2 seconds


,chemical.id,conc,plate.id,well,timepoint,value
63360,163,0.000,1,G6,t3,131.0
63361,163,0.064,2,H4,t3,25.1
63362,163,0.064,2,G4,t3,0.0
63363,163,0.064,2,F10,t3,93.1
63364,163,0.064,2,F4,t3,1.1


,chemical.id,conc,plate.id,well,timepoint,value
334075,3169,0.0,1,G12,t17,49.5
334076,3169,6.4,1,E8,t17,103.1
334077,3169,0.0,1,H6,t17,0.0
334078,3169,0.0,2,B6,t17,0.0
334079,3169,6.4,2,A2,t17,0.9


In [78]:
display("unique plate.id:" + str(np.unique(df_recasted['plate.id'])))

"unique plate.id:['1' '2' 'TP962-H10-P1']"

In [80]:
# old, uses Paritosh' groupby
### Transpose time points 
#'''
start_time = time.time()
       
def reformat(chemical_index, df_recasted, reformatted):
    df_per_chemical = df_recasted.loc[df_recasted['chemical.id'] == str(chemical_index),:]
    #print (df_per_chemical)

    # Append chemical_plate_well as a unique identifier
    # takes long time (1 min?)
    df_per_chemical.insert(0, 'chemical_plate_well', df_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(df_per_chemical.chemical_plate_well):

        #print (str(cpw))
        df_per_cpw = df_per_chemical.loc[df_per_chemical.chemical_plate_well == cpw,:]
        #display (np.unique(df_per_cpw['chemical.id']))
        df_per_cpw_grouped = df_per_cpw.groupby(['chemical.id', 'plate.id', 'well'])
        for name, group in df_per_cpw_grouped:
            #print ("name:"+str(name))
            #print ("group:"+str(group))
            concat_this = pd.DataFrame(
                    {
                    'chemical.id': np.unique(df_per_cpw['chemical.id']),
                    'plate.id': np.unique(df_per_cpw['plate.id']),
                    'well': np.unique(df_per_cpw['well']),
                    'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                    'conc': np.unique(df_per_cpw['conc'])
                    })
            
            # rename endpoint columns if this is for 15 endpoints
            for time_point in np.arange(len(np.unique(group.timepoint))):
                #print ("np.unique(group.timepoint):"+str(np.unique(group.timepoint)))
                if (len(np.unique(group.timepoint)) == 15):
                    time_point = time_point + 3
                timepoint = 't'+ str(time_point)
                #print ("\nvariable:"+str(variable))
                concat_this = pd.concat([concat_this, pd.DataFrame({timepoint: df_per_cpw.value[df_per_cpw.timepoint == timepoint].values})],axis = 1)
            reformatted = pd.concat([reformatted, concat_this])
    return reformatted
########### end of def reformat(chemical_index, df_recasted, reformatted):


reformatted = pd.DataFrame()

full_devel = "full"
#full_devel = "devel"

chemical_id_from_here = np.unique(df_behav['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 1)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

#chemical_id_from_here = ['1846']
#chemical_id_from_here = ['471']

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    print("chemical_index:" + str(chemical_index))
    reformatted = reformat(chemical_index, df_recasted, reformatted)
    
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("transposing time points is done. It took :"+str(time_took)) 
# took 5~10 minutes for 148 chemicals
display(reformatted.head())

chemical_index:163
1 chemicals processed out of 148
chemical_index:165
2 chemicals processed out of 148
chemical_index:172
3 chemicals processed out of 148
chemical_index:176
4 chemicals processed out of 148
chemical_index:197
5 chemicals processed out of 148
chemical_index:332
6 chemicals processed out of 148
chemical_index:337
7 chemicals processed out of 148
chemical_index:338
8 chemicals processed out of 148
chemical_index:339
9 chemicals processed out of 148
chemical_index:342
10 chemicals processed out of 148
chemical_index:355
11 chemicals processed out of 148
chemical_index:357
12 chemicals processed out of 148
chemical_index:362
13 chemicals processed out of 148
chemical_index:363
14 chemicals processed out of 148
chemical_index:370
15 chemicals processed out of 148
chemical_index:373
16 chemicals processed out of 148
chemical_index:380
17 chemicals processed out of 148
chemical_index:395
18 chemicals processed out of 148
chemical_index:396
19 chemicals processed out of 148
ch

,chemical.id,plate.id,well,chemical_plate_well,conc,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17
0,163,1,A1,163_1_A1,64.0000,0.9,0.0,1.3,0.0,0.3,0.1,0.0,52.0,0.0,0.0,0.0,19.9,20.8,12.6,12.3
0,163,1,A10,163_1_A10,0.0640,31.0,23.2,0.0,0.0,3.3,28.2,0.0,0.0,0.0,0.0,0.0,15.7,50.3,0.0,0.0
0,163,1,A11,163_1_A11,0.0064,4.7,30.7,60.0,38.4,34.7,0.0,26.9,8.8,56.4,9.8,37.1,1.4,50.6,0.0,4.2
0,163,1,A12,163_1_A12,0.0000,68.9,40.8,4.4,1.5,19.5,10.3,16.7,81.1,4.6,18.5,0.6,0.0,0.0,38.3,0.3
0,163,1,A2,163_1_A2,6.4000,0.1,42.9,0.9,90.6,23.4,29.0,49.4,6.2,29.7,59.2,17.5,1.2,25.0,16.3,39.7


In [81]:
reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_t3_t17_" + str(full_devel) + ".csv"
display ("reformatted_data_filename:", reformatted_data_filename)
reformatted.to_csv(reformatted_data_filename,index=False)

'reformatted_data_filename:'

'/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/after_Lisa_fix/full/15_endpoints/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_w_15_endpoints_wide_t3_t17_full.csv'

### all belows were investigational

In [74]:
# new, do not use Paritosh' groupby
### Transpose time points 

start_time = time.time()
       
def reformat(chemical_index, df_recasted, reformatted):
    df_per_chemical = df_recasted.loc[df_recasted['chemical.id'] == str(chemical_index),:]
    #print (df_per_chemical)

    # Append chemical_plate_well as a unique identifier
    # takes long time (1 min?)
    df_per_chemical.insert(0, 'chemical_plate_well', df_per_chemical.loc[:,['chemical.id','plate.id', 'well']].apply(lambda x: '_'.join(x.map(str)), axis = 1))
    
    for cpw in np.unique(df_per_chemical.chemical_plate_well):
        #print (str(cpw))
        df_per_cpw = df_per_chemical.loc[df_per_chemical.chemical_plate_well == cpw,:]
        
        concat_this = pd.DataFrame(
                {
                'chemical.id': np.unique(df_per_cpw['chemical.id']),
                'plate.id': np.unique(df_per_cpw['plate.id']),
                'well': np.unique(df_per_cpw['well']),
                'chemical_plate_well': np.unique(df_per_cpw['chemical_plate_well']),
                'conc': np.unique(df_per_cpw['conc'])
                })
    
        # Rename timepoint columns if this is for 15 endpoints
        #if(len(df_per_cpw.variable) == 15):
            
        '''
        for time_point in np.arange(len(np.unique(group.variable))):
            #print ("np.unique(group.variable):"+str(np.unique(group.variable)))
            if (len(np.unique(group.variable)) == 15):
                time_point = time_point + 3
            variable = 't'+ str(time_point)
            #print ("\nvariable:"+str(variable))
            concat_this = pd.concat([concat_this, pd.DataFrame({variable: df_per_cpw.value[df_per_cpw.variable == variable].values})],axis = 1)
        '''     
        reformatted = pd.concat([reformatted, concat_this])
    return reformatted
########### end of def reformat(chemical_index, df_recasted, reformatted):


reformatted = pd.DataFrame()

#full_devel = "full"
full_devel = "devel"

chemical_id_from_here = np.unique(df_behav['chemical.id'])

if (full_devel == "devel"):
    randomly_chosen = random.sample(set(chemical_id_from_here), 1)
    chemical_id_from_here = []
    for i in range(len(randomly_chosen)):
        chemical_id_from_here.append(randomly_chosen[i])

#chemical_id_from_here = ['1846']
#chemical_id_from_here = ['471']

total_number_of_chemicals_to_processed = len(chemical_id_from_here)
number_of_chemicals_processed = 0

for chemical_index in chemical_id_from_here:
    print("chemical_index:" + str(chemical_index))
    reformatted = reformat(chemical_index, df_recasted, reformatted)
    
    number_of_chemicals_processed += 1
    print_this = str(number_of_chemicals_processed) + " chemicals processed out of " + str(total_number_of_chemicals_to_processed)
    print(print_this)

end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("transposing time points is done. It took :"+str(time_took)) 
# took 5~10 minutes for 148 chemicals

display(reformatted.head())

chemical_index:2373
1 chemicals processed out of 1
transposing time points is done. It took :0.0 seconds


""


In [46]:
display(reformatted.head())

,chemical.id,plate.id,well,chemical_plate_well,conc,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17
0,3055,1,A1,3055_1_A1,64.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.2,90.7,86.2,119.2,108.2,90.7,116.3,57.7
0,3055,1,A10,3055_1_A10,0.0640,0.5,0.2,1.2,0.9,8.7,37.3,32.3,82.9,62.4,113.1,102.3,87.8,10.8,32.4,0.7
0,3055,1,A11,3055_1_A11,0.0064,0.0,29.2,0.0,38.4,0.1,11.0,21.7,105.9,79.6,53.5,61.4,82.9,48.1,39.9,65.8
0,3055,1,A12,3055_1_A12,0.0000,14.9,0.2,0.0,42.6,0.1,29.1,1.1,146.6,127.5,122.8,115.1,100.8,92.4,102.5,111.5
0,3055,1,A2,3055_1_A2,6.4000,18.8,0.0,11.4,1.1,0.0,0.0,0.0,158.7,160.6,132.8,121.1,106.3,136.2,122.4,114.4


In [47]:
reformatted_data_filename = str(complete_input_file_path[:-4]) + "_wide_t3_t17_" + str(full_devel) + ".csv"
display ("reformatted_data_filename:", reformatted_data_filename)
reformatted.to_csv(reformatted_data_filename,index=False)

'reformatted_data_filename:'

'/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II/input/LPR/after_Lisa_fix/full/15_endpoints/tall/344_zf_LPR_data_phase_1_2_2020JUNE25_updated_plate_id_for_TX_tall_fixed_full_w_15_endpoints_wide_t3_t17_devel.csv'

In [ ]:
'''
nan = df_select[df_select['value'].isna()]
display(nan.head())

# [phase III] there is no nan in 'chemical.id', 'conc', 'plate.id', 'well', 'variable'

display("before dropna, len(behav_select):"+str(len(df_select)))
df_select = df_select.dropna(how='any')
display("after dropna,  len(behav_select):"+str(len(df_select)))

display(df_select.head())

# phase I & II -> dropped some
# phase III    -> dropped many
'''

In [ ]:
''' # basic check of variable #
full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(behav_select['chemical.id'])
else: # full_devel = "devel"
    chemical_id_from_here = np.unique([234])
    
for chemical_index in chemical_id_from_here:
    behav_per_chemical = behav_select.loc[behav_select['chemical.id'] == chemical_index,:]
    print("chemical_index:" + str(chemical_index))

    variables = np.unique(behav_per_chemical['variable'])
    print("variables:" + str(variables))
#    variable_splited = variable.split("t")
    
 #   print (min(variable_splited[1]))
    var_len = len(np.unique(behav_per_chemical['variable']))
    if (var_len != 15):
        display("number of variable:" + str(len(np.unique(behav_per_chemical['variable']))))
'''

In [ ]:
'''df_recasted_1846 = df_recasted.loc[df_recasted['chemical.id'] == '1846',:]
display(df_recasted_1846.head())
#df_select_1846.to_csv("df_select_1846.csv",index=False)
#display(np.unique(df_select_1846['plate.id']))
display(df_recasted_1846['plate.id'])'''

In [ ]:
'''df_select_471 = df_recasted.loc[df_recasted['chemical.id'] == '471',:]
display(df_select_471.head())
#df_select_1846.to_csv("df_select_1846.csv",index=False)
display(np.unique(df_select_471['plate.id']))
'''